In [ ]:
# Ensure installation of the required packages
import sys
import subprocess
import pkg_resources

def ensure_requirements(req_file="requirements.txt"):
    """
    Check if packages from requirements.txt are installed,
    install only missing or outdated ones.
    """
    # Read requirements.txt
    with open(req_file) as f:
        required = [line.strip() for line in f if line.strip() and not line.startswith("#")]

    # Get currently installed packages
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # Find packages that need installing/upgrading
    missing_or_outdated = []
    for req in required:
        try:
            pkg_resources.require(req)
        except pkg_resources.DistributionNotFound:
            missing_or_outdated.append(req)  # Not installed
        except pkg_resources.VersionConflict:
            missing_or_outdated.append(req)  # Wrong version

    # Install only if needed
    if missing_or_outdated:
        print(f"Installing/Updating: {missing_or_outdated}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", *missing_or_outdated])
    else:
        print("✅ All required packages are already installed.")

# Run it
ensure_requirements()

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig

ModuleNotFoundError: No module named 'datasets'